In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.8.1 anndata==0.7.6 umap==0.5.1 numpy==1.20.2 scipy==1.7.1 pandas==1.2.5 scikit-learn==0.24.2 statsmodels==0.12.2 python-igraph==0.9.6 pynndescent==0.5.4


In [2]:
protein = sc.read_h5ad("/Users/bram/jointomicscomp/data/pbmc_multimodal_ADT.h5ad")

In [30]:
rna = sc.read_h5ad("/Users/bram/jointomicscomp/data/pbmc_multimodal_RNA.h5ad")

/Users/bram/anaconda3/envs/vae/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


In [4]:
protein

AnnData object with n_obs × n_vars = 161764 × 224
    obs: 'nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'lane', 'donor', 'time', 'celltype.l1', 'celltype.l2', 'celltype.l3', 'Phase'
    var: 'features'
    obsm: 'X_apca', 'X_aumap', 'X_pca', 'X_spca', 'X_umap', 'X_wnn.umap'
    varm: 'APCA'

In [38]:
if len(np.intersect1d(rna.obs.index.values, protein.obs.index.values)) == len(protein.obs.index.values):
    print("Same samples")

Same samples


In [32]:
print(rna.X.shape)
rna.obs

(161764, 20729)


,nCount_ADT,nFeature_ADT,nCount_RNA,nFeature_RNA,orig.ident,lane,donor,time,celltype.l1,celltype.l2,celltype.l3,Phase
L1_AAACCCAAGAAACTCA,7535.0,217,10823.0,2915,P2_7,L1,P2,7,Mono,CD14 Mono,CD14 Mono,G1
L1_AAACCCAAGACATACA,6013.0,209,5864.0,1617,P1_7,L1,P1,7,CD4 T,CD4 TCM,CD4 TCM_1,G1
L1_AAACCCACAACTGGTT,6620.0,213,5067.0,1381,P4_3,L1,P4,3,CD8 T,CD8 Naive,CD8 Naive,S
L1_AAACCCACACGTACTA,3567.0,202,4786.0,1890,P3_7,L1,P3,7,NK,NK,NK_2,G1
L1_AAACCCACAGCATACT,6402.0,215,6505.0,1621,P4_7,L1,P4,7,CD8 T,CD8 Naive,CD8 Naive,G1
...,...,...,...,...,...,...,...,...,...,...,...,...
E2L8_TTTGTTGGTCGTGATT,4170.0,197,9346.0,2201,P5_7,E2L8,P5,7,CD8 T,CD8 Naive,CD8 Naive,S
E2L8_TTTGTTGGTGTGCCTG,6927.0,209,9318.0,2938,P5_3,E2L8,P5,3,Mono,CD14 Mono,CD14 Mono,G1
E2L8_TTTGTTGGTTAGTTCG,4222.0,206,11619.0,3224,P8_0,E2L8,P8,0,B,B intermediate,B intermediate kappa,S
E2L8_TTTGTTGGTTGGCTAT,6063.0,217,15436.0,3999,P5_3,E2L8,P5,3,Mono,CD16 Mono,CD16 Mono,G1


In [29]:
print(protein.X.shape)
protein.obs

(161764, 224)


,nCount_ADT,nFeature_ADT,nCount_RNA,nFeature_RNA,orig.ident,lane,donor,time,celltype.l1,celltype.l2,celltype.l3,Phase
L1_AAACCCAAGAAACTCA,7535.0,217,10823.0,2915,P2_7,L1,P2,7,Mono,CD14 Mono,CD14 Mono,G1
L1_AAACCCAAGACATACA,6013.0,209,5864.0,1617,P1_7,L1,P1,7,CD4 T,CD4 TCM,CD4 TCM_1,G1
L1_AAACCCACAACTGGTT,6620.0,213,5067.0,1381,P4_3,L1,P4,3,CD8 T,CD8 Naive,CD8 Naive,S
L1_AAACCCACACGTACTA,3567.0,202,4786.0,1890,P3_7,L1,P3,7,NK,NK,NK_2,G1
L1_AAACCCACAGCATACT,6402.0,215,6505.0,1621,P4_7,L1,P4,7,CD8 T,CD8 Naive,CD8 Naive,G1
...,...,...,...,...,...,...,...,...,...,...,...,...
E2L8_TTTGTTGGTCGTGATT,4170.0,197,9346.0,2201,P5_7,E2L8,P5,7,CD8 T,CD8 Naive,CD8 Naive,S
E2L8_TTTGTTGGTGTGCCTG,6927.0,209,9318.0,2938,P5_3,E2L8,P5,3,Mono,CD14 Mono,CD14 Mono,G1
E2L8_TTTGTTGGTTAGTTCG,4222.0,206,11619.0,3224,P8_0,E2L8,P8,0,B,B intermediate,B intermediate kappa,S
E2L8_TTTGTTGGTTGGCTAT,6063.0,217,15436.0,3999,P5_3,E2L8,P5,3,Mono,CD16 Mono,CD16 Mono,G1


In [40]:
sc.pp.highly_variable_genes(rna, n_top_genes=5000)

If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:28)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


In [62]:
print(rna.var['highly_variable'])
print(rna.var['features'])

# print(len(rna.var['highly_variable'].values))
# np.sum(rna.var['highly_variable'].values)

AL627309.1    False
AL669831.5    False
LINC00115     False
FAM41C        False
NOC2L         False
              ...  
AC016588.1    False
FAM83E        False
Z82244.2      False
AP001468.1    False
AP001469.2    False
Name: highly_variable, Length: 20729, dtype: bool
AL627309.1    AL627309.1
AL669831.5    AL669831.5
LINC00115      LINC00115
FAM41C            FAM41C
NOC2L              NOC2L
                 ...    
AC016588.1    AC016588.1
FAM83E            FAM83E
Z82244.2        Z82244.2
AP001468.1    AP001468.1
AP001469.2    AP001469.2
Name: features, Length: 20729, dtype: object
